In [17]:
import os, csv, time, requests
from datetime import datetime, timedelta, timezone

API_KEY  = os.getenv("ODDS_API_KEY") or "340d84031bd365136b8dece4bc4b387e"
SPORT    = "basketball_nba"
REGION   = "us"
ODDS_FMT = "american"
DATE_FMT = "%Y-%m-%d"

# --- SMALL TEST WINDOW (adjust as needed) ---
START_DATE = "2024-01-03"   # day after your last scraped game
END_DATE   = "2024-04-14"   # regular season end

# Snapshot timing (how long before tip to pull history)
SNAP_BEFORE_MIN = 60

# NBA player prop market keys per docs
NBA_PROP_MARKETS = [
    "player_points","player_rebounds","player_assists","player_threes",
    "player_turnovers",
]

BASE = "https://api.the-odds-api.com/v4"

# --- Request/row counters ---
REQ_COUNT = 0
ROW_COUNT = 0

def get(url, params, retries=3, sleep_s=1.0):
    global REQ_COUNT
    for i in range(retries):
        r = requests.get(url, params=params, timeout=30)
        if r.status_code == 429:
            # rate limited: back off
            time.sleep(5 + i * 5)
            continue
        if r.ok:
            REQ_COUNT += 1
            return r.json(), r.headers
        # transient errors
        time.sleep(sleep_s * (i + 1))
    r.raise_for_status()

def historical_events_for_day(day_iso: str):
    """
    List historical events (game IDs) on this day (UTC snapshot at noon).
    Endpoint: /historical/sports/{sport}/events
    """
    date_ts = f"{day_iso}T12:00:00Z"
    url = f"{BASE}/historical/sports/{SPORT}/events"
    params = {"apiKey": API_KEY, "date": date_ts}
    data, _ = get(url, params)
    return data.get("data", []) if isinstance(data, dict) else []

def historical_event_props(event_id: str, snapshot_iso: str):
    """
    Get draftkings-only player prop odds for a single event at snapshot time.
    Endpoint: /historical/sports/{sport}/events/{eventId}/odds
    """
    url = f"{BASE}/historical/sports/{SPORT}/events/{event_id}/odds"
    params = {
        "apiKey": API_KEY,
        "regions": REGION,
        "oddsFormat": ODDS_FMT,
        "date": snapshot_iso,
        "markets": ",".join(NBA_PROP_MARKETS),
        "bookmakers": "draftkings",  # <<< draftkings-only
    }
    data, _ = get(url, params)
    return data.get("data", {}) if isinstance(data, dict) else {}

def choose_snapshot(commence_iso: str, minutes_before: int = 60) -> str:
    tip = datetime.fromisoformat(commence_iso.replace("Z","+00:00"))
    snap = tip - timedelta(minutes=minutes_before)
    return snap.replace(tzinfo=timezone.utc).isoformat().replace("+00:00","Z")

out_path = "nba_player_props_draftkings_sample.csv"
fieldnames = [
    "event_id","commence_time","home_team","away_team",
    "bookmaker","market_key","participant","outcome_name","price","point"
]

file_exists = os.path.isfile(out_path)

# Fresh file each run
with open(out_path, "a", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    if not file_exists:
        writer.writeheader()

    cur = datetime.fromisoformat(START_DATE).date()
    end = datetime.fromisoformat(END_DATE).date()

    while cur <= end:
        day_iso = cur.strftime(DATE_FMT)
        day_rows = 0
        try:
            events = historical_events_for_day(day_iso)
        except Exception as e:
            print(f"[{day_iso}] events error: {e}")
            cur += timedelta(days=1)
            continue

        for ev in events:
            try:
                event_id = ev["id"]
                commence = ev["commence_time"]
                home = ev.get("home_team")
                away = ev.get("away_team")
                snap_iso = choose_snapshot(commence, SNAP_BEFORE_MIN)

                ev_data = historical_event_props(event_id, snap_iso)
                if not ev_data:
                    continue

                bookmakers = ev_data.get("bookmakers", [])
                for bk in bookmakers:
                    bk_name = bk.get("key")
                    if bk_name != "draftkings":
                        continue  # defensive: keep only draftkings
                    for m in bk.get("markets", []):
                        mkey = m.get("key")
                        if mkey not in NBA_PROP_MARKETS:
                            continue
                        for oc in m.get("outcomes", []):
                            row = {
                                "event_id": event_id,
                                "commence_time": commence,
                                "home_team": home,
                                "away_team": away,
                                "bookmaker": bk_name,
                                "market_key": mkey,
                                "participant": oc.get("name"),
                                "outcome_name": oc.get("description") or oc.get("name"),
                                "price": oc.get("price"),
                                "point": oc.get("point"),
                            }
                            writer.writerow(row)
                            ROW_COUNT += 1
                            day_rows += 1

                time.sleep(0.2)  # be polite
            except Exception as e:
                print(f"[{day_iso}] event {ev.get('id')} error: {e}")
                continue

        print(f"Done {day_iso} | events: {len(events)} | draftkings prop rows: {day_rows}")
        time.sleep(0.25)
        cur += timedelta(days=1)

print(f"\nRequests used: {REQ_COUNT} | Rows written: {ROW_COUNT} → {out_path}")

Done 2024-01-03 | events: 12 | draftkings prop rows: 1219
Done 2024-01-04 | events: 2 | draftkings prop rows: 252
Done 2024-01-05 | events: 14 | draftkings prop rows: 1346
Done 2024-01-06 | events: 4 | draftkings prop rows: 392
Done 2024-01-07 | events: 9 | draftkings prop rows: 840
Done 2024-01-08 | events: 6 | draftkings prop rows: 650
Done 2024-01-09 | events: 5 | draftkings prop rows: 522
Done 2024-01-10 | events: 10 | draftkings prop rows: 1012
Done 2024-01-11 | events: 5 | draftkings prop rows: 548
Done 2024-01-12 | events: 10 | draftkings prop rows: 1108
Done 2024-01-13 | events: 8 | draftkings prop rows: 830
Done 2024-01-14 | events: 5 | draftkings prop rows: 280
Done 2024-01-15 | events: 11 | draftkings prop rows: 1084
Done 2024-01-16 | events: 3 | draftkings prop rows: 372
[2024-01-17] event 36e7461f05a993188963028c3eef01ec error: 404 Client Error: Not Found for url: https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events/36e7461f05a993188963028c3eef01ec/odds?